In [1]:
import numpy as np
import matplotlib.pyplot as plt
import copy
import random
import time
random.seed(time.time())

## Exercise 1 (SIR with DTMC in ABM)

In this exercise we will start to work with some unhomogenous matrix. This type of problems we will allow us to present another important type of simulations.

Generally speaking there are two huge macro approach to the simulations in systems: the SD and the ABM. For the SD the book Thinking in Systems can be useful. However certain problem can be described using both the two approach while other can be written using only one of the two approches. Some problems are even an hybrid between two of them. If you want, at the end of this course I can do a small lecture about these two approaches. 

However, the simulations of all these kind of systems can be done again with one of the two methods and sometimes the implentation and simulation method will not coincide. 

We want to take in consideration the classical SIR problem.

**SIR**: the SIR model is the most basic mathematical model to describe the behaviour of an epidemy. It works pretty well with malaria and it's used as base model to most complex epidemic model. According to the SIR, during an epidemy, all the population can be categorized in three classes: Susceptible, Infect, Removed. At each time instant we have that a person can pass from a class to another with a certain probability. The problem is that this probability is not fixed in time but depends on how many subjects there are for each category (es. the infection of a subsceptible is proportionaly frequent to the number of infects).
We have the following defined transition

- S -> I
- I -> R

This POV is based on a ABM approach but this problem can be also treated in SD considering not the single person but the number of subjects in each category.

Now we want to model a DTMC that will tell us given the starting probability for a person to be in each one of the categories the probability vector after a certain time of iterations $n$. Obviously the scenario we will change according to distribution of the population between the three categories at the beginning of the simulation. 

The general idea to solve this problem is the subsequent. We will compute given a probability distribution using the matrix method. To compute the matrix we will performed a simulation with 1000 Agents that form the population that will evolve. This method allows us to consider a different initial probability distribution for the population and for our simulation. 
Technically doing this is as having a single person with an initial probability of being in one of the three states and look how it evolves. Obviously having different behaviours for the population we will need also to repeat this experiment multiple time and consider the mean probability distribution. 

This is not the only simulative method. There are different possibilities modelling different aspects. It's also theoretically possible to consider the population in a SD approach and evolve it. Obviously in this situation the equations will be different and computation of the matrix will be based on different mathematical operations. It's also possible to consider the population distribution and evolve it separately using the matrix and produce the newest with the new distribution.

In [2]:
#We will suppose that the given population distribution multiplied for 1000 will produce exact integers. Otherwise some clamping and rounding
#methods have to be implemented.

class DTMC_SIR(object):
    
    #COMPUTE CUMULATIVE PROBABILITY ARRAY
    def __cumulativeTransformer(self,p0:np.array)->np.array:
        '''
        Given a probability array this function will compute
        the cumulative of such distribution.
        '''
        p = np.zeros(len(p0),dtype=float)
        p[0] = p0[0]

        for i in range(1,len(p)):
            p[i] = p0[i] + p[i-1]
        
        return p

    #INITIAL STATE COMPUTING
    def __obtainState(self,p0:np.array)->int:
        '''
        Given a probability distribution of the state this function will
        produce a weighted random state.
        '''
        p = self.__cumulativeTransformer(p0)

        r = random.random()
        s = 0
        
        for i in range(len(p0)):
            if(i!=0):
                if((r<p[i]) and ((p[i]-p[i-1])>0.0000001)):
                    s = i
                    break
            else:
                if r<p[i]:
                    s = i
                    break
        
        return s

    #The transition probability for our single
    def __createProbTransitions(self,S: int)->np.array:
        '''
        Given the starting state. The method will produce the probability
        distribution of the transitions
        '''

    #EVOLVE THE POPULATION
    def __evolvePopulation(self)->None:
        '''
        This method will evolve the population and recalculate
        __pop_dist
        '''

    #SIMULATION METHOD
    def __simulateTraj(self,p0:np.array,time: int)->int:
        '''
        This function will simulate a trajectory of the DTMC_SIR
        '''
        state = self.__obtainState(p0)

        for i in range(time):
            p = self.__createProbTransitions(state)
            state = self.__obtainState(p)
            self.__evolvePopulation()

        return state
    
    #CONSTRUCTOR
    def __init__(self,pdist: np.array)->None:
        self.n_states = 3
        self.__npop = 1000
        self.__origin_dist = pdist
        self.__pop_dist = copy.deepcopy(pdist)
        return

        
    #MAIN PREDICTION METHOD
    def compute_pred_forward(self, p0: np.array, time: int, N: int)->np.array:
        '''
        Calculate, given an initial probability array p0 of the states
        what will be the probability array after "time" interactions.
        N simulation will be performed.
        '''
        #if time=0 no projections have to be made
        if(time==0):
            return p0
    
        #if time is negative it as no sense 
        if(time<0):
            raise RuntimeError("Negative time for forward prediction")

        #Check N
        if(N<0):
            raise RuntimeError("Negative number of simulations")
    
        #Create the bins vector
        bins = np.zeros(self.n_states,dtype=float)
        
        for i in range(N):
            self.__pop_dist = copy.deepcopy(self.__origin_dist)
            s = self.__simulateTraj(p0,time)
            bins[s] += 1.0

        #Normalization
        bins = bins/N
        
        return bins

## Exercise 2 (Cross influence between CTMCs)

In the second part of this lecture we will treat a very generic and vague problem that can appear simple and trivial initially. We will model it using the simulative approach and the SSA trying as always to obtain the probability distribution after a certain time.

PROBLEM: We have two objects where each can be in one of three states, namely $[0,1,2]$, each of them can move to a state to another with certain transitions rates. Each one of the two objects can be modelled as a CTMC. However the transition rate for an object from a state to another depends on the state of the other object. Let's suppose for example that the object $A$ is in state $0$ and the $B$ is in state $1$. The transition rate for $A$ from 0 to 1 will be equal to 0.1 while the rate from 0 to 2 will be $1.0$. 

**What is the main issue or problem to implement this?**

The time problem! If we were in DTMC the situation would be much easier but now we are not able to compute indipendently the SSA of the two CTMC because we need the time syncronization between the two processes!

As it turns out, this trivial and oversimplified problem encapsulate one of the major issues when we have to work with multiple linked CTMC. This causistic it's more common than percieved: e.g. we have the problem of the office workers (Assignment 1) but each worker has it's own queue with arrival probability dependent by the number of tasks in the worker's queue.

In [3]:
class DOUBLE_CTMC_SIM(object):
    '''
    Continuous-Time Markov Chain Class
    based on a simulative approach.
    '''

    @staticmethod
    def generator_check(M: np.matrix)->None:
        '''
        This method will check if the given generator
        is valid or not.
        '''

        #Let's check first if it's 2D
        if(M.ndim!=2):
            raise RuntimeError("The infinitesimal generator have to be 2D")

        #2. Check if it's a squared matrix
        if(np.shape(M)[0]!=np.shape(M)[1]):
            raise RuntimeError("Number of rows is different from the number of columns")

        #3. Check if the out of diagonal coefficient are positive and if the row sum up to zero
        #NOTE: with the positivity request is equivalent to ask the sum up to the diagonal coeff.
        for i in range(np.shape(M)[0]):
            s = 0
            for j in range(np.shape(M)[1]):
                if(M[i,j]<0 and i!=j): #Out of diagonal
                    print(i,j)
                    raise RuntimeError("The infinitesimal generator has a negative entry")
                s=s+M[i,j]
            if(s>0.0000001): #Row sum up
                print(i)
                raise RuntimeError("The infinitesimal generator has a row that does not sum up to 0")

        return

    @staticmethod
    def jump_chain_mat_generator(Q: np.matrix)->np.matrix:
        '''
        This function will take a supposed valid
        infinitesimal generator and will produce
        the transition matrix for the associated
        jump chain.
        '''

        n = np.shape(Q)[0]
        J = np.zeros((n,n),dtype=float)

        for i in range(n):
            a = -Q[i,i]
            for j in range(n):
                if(i!=j):
                    J[i,j]=Q[i,j]/a

            J[i,i]=0

        return J

    @staticmethod
    def exp_time(l: float)->float:
        '''
        Given the rate, this function will generate a random time
        exponentially distributed
        '''
        r = random.random()
        n = -np.log(1-r)/l

        return n

    #COMPUTE CUMULATIVE PROBABILITY ARRAY
    def __cumulativeTransformer(self,p0:np.array)->np.array:
        '''
        Given a probability array this function will compute
        the cumulative of such distribution.
        '''
        p = np.zeros(len(p0),dtype=float)
        p[0] = p0[0]

        for i in range(1,len(p)):
            p[i] = p0[i] + p[i-1]
        
        return p

    #STATE COMPUTATION
    def __obtainState(self,p0:np.array)->int:
        '''
        Given a probability distribution of the state this function will
        produce a weighted random state.
        '''
        p = self.__cumulativeTransformer(p0)

        r = random.random()
        s = 0
        
        for i in range(len(p0)):
            if i!=0:
                if((r<p[i]) and (abs(p[i]-p[i-1])>0.0000001)):
                    s = i
                    break
            else:
                if r<p[i]:
                    s = i
                    break
        return s

    #ABOVE THE SAME OLD STUFF

    def __compute_infinitesimal_generator(self,S: int)->np.matrix:
        '''
        Given the state of the other object this function will compute
        the infinitesimal generator for the object.
        '''

    def __simulationSSADouble(self,p0: np.array,T: float)->np.array:
        '''
        This function will use the Gillespie-SSA algorithm
        to produce a stochastic trajectory for the CTCM
        '''

    def compute_prob_dist(self,p0:np.array,T: float,Nsim: int):
        '''
        Given the initial distribution this function will produce
        the distribution after time T using Nsim simulations
        '''

    #CONSTRUCTOR
    def __init__(self)->None:
        self.n_states= 3
        return